In [ ]:
#this is to be removed ... this is just a dummy model to test the pipeline
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define the model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 3)  # Assuming 3 classes: bored, attentive, confused

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Dummy training loop (Replace with real data)
for epoch in range(1):  # Run real training loop instead
    optimizer.zero_grad()
    dummy_input = torch.randn(1, 3, 224, 224)
    output = model(dummy_input)
    loss = loss_fn(output, torch.tensor([1]))  # Example target class
    loss.backward()
    optimizer.step()

# Save trained model
torch.save(model.state_dict(), "../models/model.pth")
print("✅ Model saved successfully at models/model.pth")


✅ Model saved successfully at models/model.pth


### Define the DAiSEEDataset Class

This cell defines the full custom dataset class for DAiSEE. This definition is required for loading the saved dataset checkpoints and for training the model. Make sure to run this cell before loading any checkpoints.

Below is the code:


In [6]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
from pathlib import Path
from PIL import Image
import logging
from tqdm import tqdm
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image

# Ensure the logger shows DEBUG messages
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

class DAiSEEDataset(Dataset):
    """
    Updated dataset class for DAiSEE that reads a CSV with four engagement metrics.
    It precomputes the frame paths based on the ClipID and loads labels for 
    Boredom, Engagement, Confusion, and Frustration.
    
    If the ClipID does not contain a slash (i.e. it's just a number), the split is
    inferred from the CSV file name. For example, for TrainLabels.csv, we assume the split is "Train".
    """
    def __init__(self, root, csv_path, transform=None, seq_length=15):
        self.root = Path(root)
        self.transform = transform
        self.seq_length = seq_length
        
        # Read the CSV file
        self.data = pd.read_csv(csv_path)
        
        # Pre-compute frame paths and labels with a progress bar
        self.video_paths = []  # List to store sequences of frame paths
        self.labels = []  # List to store labels
        
        logger.info(f"Processing {len(self.data)} entries from {csv_path.name}")
        for idx, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Loading dataset"):
            try:
                clip_id = str(row['ClipID']).strip()
                # Remove the .avi extension if present
                if '.avi' in clip_id:
                    clip_id = clip_id.replace('.avi', '')
                parts = clip_id.split('/')
                
                # Determine the frame path based on the number of parts:
                if len(parts) == 1:
                    # If only one part exists, infer the split from the CSV filename.
                    # For example, if csv_path is TrainLabels.csv, we deduce split = "Train".
                    split_guess = csv_path.stem.replace("Labels", "").strip()
                    video_dir = self.root / split_guess / parts[0]
                elif len(parts) == 2:
                    video_dir = self.root / parts[0] / parts[1]
                elif len(parts) >= 3:
                    video_dir = self.root / parts[0] / parts[1] / parts[2]
                else:
                    logger.debug(f"Skipping row {idx}: unexpected ClipID format: {clip_id}")
                    continue  # Skip rows that do not match the expected pattern

                # Ensure the video directory exists
                if not video_dir.exists():
                    logger.debug(f"Video directory does not exist: {video_dir}")
                    continue

                # Collect frame paths
                frames = list(video_dir.glob('frame_*.jpg'))
                if len(frames) < self.seq_length:
                    logger.debug(f"Skipping video {clip_id}: not enough frames ({len(frames)} < {self.seq_length})")
                    continue

                frames.sort()  # Ensure chronological order
                self.video_paths.append(frames[:self.seq_length])  # Take the first `seq_length` frames
                self.labels.append([
                    float(row['Boredom']),
                    float(row['Engagement']),
                    float(row['Confusion']),
                    float(str(row['Frustration ']).strip())
                ])
            except Exception as e:
                logger.debug(f"Skipping row {idx} due to error: {e}")
                continue
        
        logger.info(f"Loaded {len(self.video_paths)} valid video sequences")
        
        if len(self.video_paths) == 0:
            raise ValueError(f"No valid video sequences found in {csv_path}")
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        frames_paths = self.video_paths[idx]
        labels = self.labels[idx]
        
        frame_tensors = []
        for path in frames_paths:
            img = Image.open(path)
            if self.transform:
                img = self.transform(img)
            frame_tensors.append(img)
        
        sequence = torch.stack(frame_tensors)  # [seq_len, C, H, W]
        return sequence, torch.tensor(labels, dtype=torch.float32)
    
# Register the dataset class for safe deserialization.
torch.serialization.add_safe_globals({"DAiSEEDataset": DAiSEEDataset})

### Load Saved Datasets

This cell loads the full datasets that were saved in 001_data.ipynb (e.g., `train_set.pth`, `val_set.pth`, and `test_set.pth`). Make sure to run this cell before training the model.


In [9]:
import torch
import os
from torch.utils.data import DataLoader

# Define the path where the dataset checkpoints were saved.
FRAMES_ROOT = "C:/Users/abhis/Downloads/Documents/Learner Engagement Project/data/DAiSEE/ExtractedFrames"

# Load the saved datasets using the full DAiSEEDataset class.
train_dataset = torch.load(os.path.join(FRAMES_ROOT, "train_set.pth"), weights_only=False)
val_dataset = torch.load(os.path.join(FRAMES_ROOT, "val_set.pth"), weights_only=False)
test_dataset = torch.load(os.path.join(FRAMES_ROOT, "test_set.pth"), weights_only=False)

print("✅ Datasets loaded successfully!")


✅ Datasets loaded successfully!


### Create DataLoaders

This cell creates DataLoaders for the training, validation, and testing datasets. These loaders will feed batches of data to your ML/DL models.


In [10]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
NUM_WORKERS = 4  # On Windows, consider using 0 if you encounter issues

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("✅ DataLoaders created successfully!")

✅ DataLoaders created successfully!
